# Import Required Packages

In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# Load Win Percentage Data

In [2]:
game_table = pd.read_csv("Datasets/win_prob_nfl_2016.csv", index_col="GAME_ID")
game_table

,WEEK,TEAM,P_WIN
GAME_ID,,,
0,1,CAR,0.40
1,1,BUF,0.47
2,1,CHI,0.33
3,1,CIN,0.49
4,1,CLE,0.29
5,1,GB,0.68
6,1,MIN,0.71
7,1,OAK,0.40
8,1,SD,0.19


# Setup Data in Linear Program Form

In [18]:
# Set Parameters
teams = game_table.TEAM.unique()
numb_teams = len(teams)
weeks = max(game_table.WEEK)
discount_rate = 0.30


# Set Objective Function Coefficients (based on discounted win probabilities)
p_win = np.array((game_table.P_WIN*((1-discount_rate)**(game_table.WEEK-1))))


# Constraint 1: Pick only one team per week
xa1 = np.zeros((weeks,numb_teams*weeks))
xb1 = np.ones((weeks,1))

i=1
for row in xa1:
    row[(i-1)*32:i*32] = 1
    i +=1
    
    
# Constraint 2: Pick each team at most once
xa2 = np.zeros((numb_teams,numb_teams*weeks))
xb2 = np.ones((numb_teams,1))

j=0
for team in teams:
    test = game_table.index[game_table.TEAM == team]
    for k in test:
        xa2[j][k] = 1
    j+=1
    
# Bounds Constraints:
x_lower_upper = (0,1)

# Arrange in Standard Form and Compute

In [19]:
c = p_win
A = np.vstack((xa1,xa2))
b = np.vstack((xb1,xb2))

res = linprog(-c, A_ub=A, b_ub=b, bounds=x_lower_upper, options={"disp": True})

Optimization terminated successfully.
         Current function value: -2.686126   
         Iterations: 35


# Print Summary Table

In [20]:
predictions = game_table.iloc[np.where(res.x == 1)[0]]
continuing = 1.0
continuing_list = []
for row in predictions.iterrows():
    continuing *= row[1]["P_WIN"]
    continuing_list.append(continuing)
    
predictions["P_CONTINUING"] = continuing_list
predictions

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,WEEK,TEAM,P_WIN,P_CONTINUING
GAME_ID,,,,
24,1,KC,0.81,0.810000
56,2,DET,0.81,0.656100
90,3,SEA,0.82,0.538002
119,4,HOU,0.81,0.435782
155,5,CAR,0.81,0.352983
187,6,GB,0.78,0.275327
211,7,CIN,0.83,0.228521
246,8,DEN,0.83,0.189673
279,9,SD,0.72,0.136564
